# Accessing data from a website
Not all websites make it easy to grab data. Luckily, `pandas` can help.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

result = requests.get('https://en.wikipedia.org/wiki/List_of_sovereign_states')
pd.read_html(result.content)[0].head(20)

,Common and formal names,Membership within the UN System[a],Sovereignty dispute[b],Further information on status and recognition of sovereignty[d]
0,NaN,NaN,NaN,NaN
1,UN member states and observer states ↓,NaN,NaN,NaN
2,Abkhazia,NaN,NaN,NaN
3,Afghanistan – Islamic Republic of Afghanistan,UN member state,NaN,NaN
4,Albania – Republic of Albania,NaN,NaN,NaN
5,Algeria – People's Democratic Republic of Algeria,NaN,NaN,NaN
6,Andorra – Principality of Andorra,NaN,NaN,Andorra is a co-principality in which the offi...
7,Angola – Republic of Angola,NaN,NaN,NaN
8,Antigua and Barbuda,NaN,NaN,Antigua and Barbuda is a Commonwealth realm[e]...
9,Argentina – Argentine Republic[g],NaN,NaN,Argentina is a federation of 23 provinces and ...


For more complex parsing, we can utilize the `BeautifulSoup` library. Let's try to extract the same table, but use the new library. 

In [2]:
soup = BeautifulSoup(result.content, 'lxml') # Parse the HTML as a string
str(soup)[:500]

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n<head>\n<meta charset="utf-8"/>\n<title>List of sovereign states - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqsP3wpAAEUAAEGqjmIAAAAR","wgCS'

Find the tables.

In [3]:
tables = soup.find_all('table')

Using the `read_html` function of `pandas`, read the first table into a dataframe.

In [4]:
pd.read_html(str(tables[0]))[0].head(20)

,Common and formal names,Membership within the UN System[a],Sovereignty dispute[b],Further information on status and recognition of sovereignty[d]
0,NaN,NaN,NaN,NaN
1,UN member states and observer states ↓,NaN,NaN,NaN
2,Abkhazia,NaN,NaN,NaN
3,Afghanistan – Islamic Republic of Afghanistan,UN member state,NaN,NaN
4,Albania – Republic of Albania,NaN,NaN,NaN
5,Algeria – People's Democratic Republic of Algeria,NaN,NaN,NaN
6,Andorra – Principality of Andorra,NaN,NaN,Andorra is a co-principality in which the offi...
7,Angola – Republic of Angola,NaN,NaN,NaN
8,Antigua and Barbuda,NaN,NaN,Antigua and Barbuda is a Commonwealth realm[e]...
9,Argentina – Argentine Republic[g],NaN,NaN,Argentina is a federation of 23 provinces and ...


As we can see, the data we get back isn't always perfect, which is what's so nice about APIs instead of parsing HTML. Nevertheless, we would benefit a lot if we simplified this into a function.

In [5]:
def dfFromURL(url, tableNumber=1):
    soup = BeautifulSoup(requests.get(url).content, 'lxml') # Parse the HTML as a string
    tables = soup.find_all('table')
    # check table number is within number of tables on the page
    assert len(tables) >= tableNumber
    return pd.read_html(str(tables[tableNumber-1]))[0]

Now we can make a pretty simple call to get an HTML table as a dataframe. Let's try it.

In [6]:
prices = dfFromURL('https://finance.yahoo.com/quote/JPM/history?p=JPM')
prices.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Apr 30, 2020",95.60,96.84,94.55,95.76,95.76,17262125
1,"Apr 29, 2020",99.08,99.45,97.27,97.86,97.86,20427700
2,"Apr 28, 2020",97.81,98.36,95.03,95.29,95.29,20238900
3,"Apr 27, 2020",91.94,95.63,91.54,94.62,94.62,22870200
4,"Apr 24, 2020",90.64,91.10,88.49,90.71,90.71,15815300


Got some messy data hear with divs and some disclaimers on the bottom...let's clean it up with a simple `dropna`.

In [7]:
prices = prices.dropna()
prices.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Apr 30, 2020",95.60,96.84,94.55,95.76,95.76,17262125
1,"Apr 29, 2020",99.08,99.45,97.27,97.86,97.86,20427700
2,"Apr 28, 2020",97.81,98.36,95.03,95.29,95.29,20238900
3,"Apr 27, 2020",91.94,95.63,91.54,94.62,94.62,22870200
4,"Apr 24, 2020",90.64,91.10,88.49,90.71,90.71,15815300


Cool! Let's try to get the second table from a website. Let's see what the Cavs record was for the last few seasons:
    

In [17]:
df1 = dfFromURL('https://www.espn.com/nba/team/stats/_/name/cle', 0)
df1

,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,7.9,16.7,47.2,1.5,3.9,38.0,3.5,4.1,84.6,6.4,12.8,50.1,1.242,0.52
1,5.8,13.0,45.0,2.6,7.0,37.4,3.3,3.9,85.4,3.2,6.0,53.7,1.358,0.55
2,7.3,13.1,55.2,0.5,1.8,28.6,2.5,4.9,51.3,6.8,11.4,59.3,1.333,0.57
3,5.0,11.2,44.2,2.0,5.5,37.1,2.6,3.0,88.4,2.9,5.8,50.9,1.298,0.53
4,4.7,11.8,40.1,1.8,5.0,35.5,1.1,1.2,87.5,3.0,6.8,43.4,1.041,0.48
5,5.1,9.9,51.2,0.2,0.4,39.1,1.7,2.8,61.5,4.9,9.5,51.8,1.217,0.52
6,4.1,9.4,43.7,1.9,4.9,38.3,1.0,1.4,67.0,2.2,4.4,49.7,1.179,0.54
7,4.1,7.7,53.1,1.0,2.8,35.2,0.9,1.3,67.6,3.1,4.9,63.6,1.309,0.60
8,3.7,8.4,44.2,1.1,3.2,33.5,1.5,2.0,72.3,2.7,5.3,50.6,1.183,0.50
9,1.9,4.0,47.9,0.5,1.5,35.1,1.3,1.7,73.2,1.4,2.5,55.9,1.406,0.55


In [18]:
df2 = dfFromURL('https://www.espn.com/nba/team/stats/_/name/cle', 1)
df2

,Name
0,Collin Sexton PG
1,Kevin Love PF
2,Andre Drummond C *
3,Jordan Clarkson PG *
4,Darius Garland PG
5,Tristan Thompson C
6,Cedi Osman SF
7,Larry Nance Jr. PF
8,Kevin Porter Jr. SG
9,Dante Exum PG *


In [21]:
pd.concat((df2, df1), axis=1)

,Name,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,2PM,2PA,2P%,SC-EFF,SH-EFF
0,Collin Sexton PG,7.9,16.7,47.2,1.5,3.9,38.0,3.5,4.1,84.6,6.4,12.8,50.1,1.242,0.52
1,Kevin Love PF,5.8,13.0,45.0,2.6,7.0,37.4,3.3,3.9,85.4,3.2,6.0,53.7,1.358,0.55
2,Andre Drummond C *,7.3,13.1,55.2,0.5,1.8,28.6,2.5,4.9,51.3,6.8,11.4,59.3,1.333,0.57
3,Jordan Clarkson PG *,5.0,11.2,44.2,2.0,5.5,37.1,2.6,3.0,88.4,2.9,5.8,50.9,1.298,0.53
4,Darius Garland PG,4.7,11.8,40.1,1.8,5.0,35.5,1.1,1.2,87.5,3.0,6.8,43.4,1.041,0.48
5,Tristan Thompson C,5.1,9.9,51.2,0.2,0.4,39.1,1.7,2.8,61.5,4.9,9.5,51.8,1.217,0.52
6,Cedi Osman SF,4.1,9.4,43.7,1.9,4.9,38.3,1.0,1.4,67.0,2.2,4.4,49.7,1.179,0.54
7,Larry Nance Jr. PF,4.1,7.7,53.1,1.0,2.8,35.2,0.9,1.3,67.6,3.1,4.9,63.6,1.309,0.60
8,Kevin Porter Jr. SG,3.7,8.4,44.2,1.1,3.2,33.5,1.5,2.0,72.3,2.7,5.3,50.6,1.183,0.50
9,Dante Exum PG *,1.9,4.0,47.9,0.5,1.5,35.1,1.3,1.7,73.2,1.4,2.5,55.9,1.406,0.55
